In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json # to read json
from datasets import load_dataset,Dataset,get_dataset_config_names
import warnings # to ignore warnings
warnings.filterwarnings('ignore')

## MLQA Format

`{split}-context-{context_language}-question-{question_language}.json`


In [153]:
context_language = 'hi'
question_language = 'en'

In [154]:
mlqa = load_dataset("mlqa",f"mlqa.{context_language}.{question_language}")

Dataset mlqa downloaded and prepared to C:\Users\Subha\.cache\huggingface\datasets\mlqa\mlqa.hi.en\1.0.0\224fde9ea61350ffb013e4beff31d44c6e125ce82c3aa4af70298eceabc8f7f7. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 92.62it/s]


In [155]:
mlqa

DatasetDict({
    test: Dataset({
        features: ['context', 'question', 'answers', 'id'],
        num_rows: 4918
    })
    validation: Dataset({
        features: ['context', 'question', 'answers', 'id'],
        num_rows: 507
    })
})

In [156]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained("../squad-qa-minilmv2-XLMTokeinizer-8/checkpoint-7000")
tokenizer = AutoTokenizer.from_pretrained("../squad-qa-minilmv2-XLMTokeinizer-8/checkpoint-7000")

loading configuration file ../squad-qa-minilmv2-XLMTokeinizer-8/checkpoint-7000\config.json
Model config BertConfig {
  "_name_or_path": "../squad-qa-minilmv2-XLMTokeinizer-8/checkpoint-7000",
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "XLMRobertaTokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 250037
}

loading weights file ../squad-qa-minilmv2-XLMTokeinizer-8/checkpoint-7000\pytorch_model.bin
All model checkpoint weights were used when initializing BertForQuestionA

In [157]:
pad_on_right = tokenizer.padding_side == "right"
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # Th

In [158]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [159]:
mlqa.column_names

{'test': ['context', 'question', 'answers', 'id'],
 'validation': ['context', 'question', 'answers', 'id']}

In [160]:
validation_features = mlqa.map(
    prepare_validation_features,
    batched=True,
    remove_columns=mlqa["test"].column_names
)

100%|██████████| 1/1 [00:02<00:00,  2.67s/ba]


In [161]:
validation_features['test']

Dataset({
    features: ['input_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 6319
})

In [162]:
# Load model into Huggingface Trainer
from transformers import Trainer
trainer = Trainer(model = model)

raw_predictions = trainer.predict(validation_features['test'])

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
100%|██████████| 933/933 [04:10<00:00,  3.72it/s]
The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping. If example_id, offset_mapping are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 6319
  Batch size = 8
100%|█████████▉| 789/790 [01:06<00:00, 11.39it/s]

In [163]:
validation_features['test'].set_format(type=validation_features['test'].format["type"], columns=list(validation_features['test'].features.keys()))

In [164]:
max_answer_length = 30

In [165]:
import collections

examples = mlqa['test']
features = validation_features['test']

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [166]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                        or offset_mapping[start_index] == []
                        or offset_mapping[end_index] == []
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        # if not squad_v2:
            # predictions[example["id"]] = best_answer["text"]
        # else:
        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer

    return predictions

In [167]:
final_predictions = postprocess_qa_predictions(mlqa['test'], validation_features['test'], raw_predictions.predictions)

100%|██████████| 790/790 [01:25<00:00, 11.39it/s]

Post-processing 4918 example predictions split into 6319 features.


100%|██████████| 4918/4918 [00:24<00:00, 200.14it/s]


In [168]:
formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in mlqa['test']]

In [169]:
# for each item in formatted predictions create a dictionary with the id as key and the prediction_text as value
prediction_dict = {p["id"]: p["prediction_text"] for p in formatted_predictions}
prediction_dict

{'eeb8dbd25efe5221dc6723ddee95daa07d2c8478': '',
 'ba7865d50777f2b90ba88fcb070a672d042b6b69': '',
 '2079cf7ce47961738e4bd0d527d0b1058210f869': 'एक आदर्श पट्टी',
 'd5377da63e6f64dae5e269290a6334c2a912cb3f': 'Douglas DC-3',
 '03df1f92420416844575cfa201ae840319c40650': 'अनुशासनात्मक',
 '7390b083695d1b6d8bed9acb57a84707361c9104': 'प्रतिबंधित',
 'a4968ca8a18de16aa3859be760e43dbd3af3fce9': 'रुटगर्स विश्वविद्यालय के जीव रसायन विज्ञानियों',
 'f251ea56c4f1aa1df270137f7e6d89c0cc1b6ef4': 'जोनाथन टर्ली',
 '04ecd5555635bc05fd2f379d1b9027edd663cebf': 'जब ग्रूम के खुले गड्ढों और खाइयों में विशाल मात्रा में अज्ञात रसायनों को जलाया गया तो वे वहां उपस्थित थे',
 'd066a75dbe8cd3e2b57c415a8eb54a08dc7e72a7': 'जब ग्रूम के खुले गड्ढों और खाइयों में विशाल मात्रा में अज्ञात रसायनों को जलाया गया तो वे वहां उपस्थित थे',
 'c5f545baccd8ea8adb83f75756f4832340600bd9': 'जनवरी सन् 2006',
 'f9307229512ff964a59eaa408e62e05c3d10398b': 'एरिया 51',
 '2a47396020fe7152a5c91c7f17bb1da798925c77': 'उपार्जित सेवा गुणवत्ता',
 'c56

In [170]:
with open(f'./formatted_predictions_{context_language}_{question_language}_baseline.json', 'w') as f:
    json.dump(prediction_dict, f)

## Run MLQA Evaluation using:
-   ARG1: Test Data Path 
-   ARG2: Predictions Path
-   ARG3: Answer Language

`python MLQA/mlqa_evaluation_v1.py ./MLQA/Data/test/test-context-en-question-hi.json ./MLQA/formatted_predictions_en_hi_baseline.json en`